# Dependents

In [ ]:
%pylab inline
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
from MP.mat import voigt
ijv=voigt.ijv
vij=voigt.vij


p_home = os.getcwd()
p_load = '/Users/yj/repo/vpsc/vpsc-dev-yld/src/py_pack/yld_hah/'

os.chdir(p_load)
import lib_dat
os.chdir(p_home)

In [ ]:
def ys_temp(ax):
    ax.grid()
    ax.set_aspect('equal')
    ax.set_xlabel(r'$\Sigma_\mathrm{11}$',fontsize=17)
    ax.set_ylabel(r'$\Sigma_\mathrm{22}$',fontsize=17)
    
def pi_proj(sd):
    """
    Deviatoric stress to pi-plane projection
    """
    sq6 = np.sqrt(6.)
    sq2 = np.sqrt(2.)    
    x = 2.*sd[0]/sq6 - sd[1] / sq6 - sd[2] / sq6
    y =                sd[1] / sq2 - sd[2] / sq2
    return x,y 

def devit(x,p=0.):
    """
    Convert it to a deviatoric space with
    hydrostatic stress of <p>
    
    Argument
    --------
    x  : Conver stress <x> to a deviatric
    p  : pressure (optional, if non-zero,
                   translate the stress along hydrostatic axis)
    """
    x=np.array(x,dtype='float')
    m=x[:3].sum()/3.
    x[:3]=x[:3]-m
    
    if p!=0:
        x[:3]=x[:3]+p/3.    
    return x

## General Von Mises stress in 6D stress space.

$\phi^\mathrm{VM}(\mathbf{\sigma}) = [\frac{1}{2}\{(\sigma_{11}-\sigma_{22})^2+(\sigma_{22}-\sigma_{33})^2+(\sigma_{33}-\sigma_{11})^2+6(\sigma_{12}^2+\sigma_{23}^2+\sigma_{31}^2)\}]^\frac{1}{2}$

In [ ]:
def vm_gen(sigma):
    """
    Von Mises yield locus general
    
    Returns phi^VM
    ----------
    sigma <6 dimensional>
    """
    s11,s22,s33,s31,s23,s12=sigma[0],sigma[1],sigma[2],\
               sigma[3],sigma[4],sigma[5]
    
    y=(s11-s22)**2.+(s22-s33)**2.+(s33-s11)**2.
    y = y + 6.*(s12**2+s23**2+s31**2)
    y = np.sqrt(0.5*y)
    return y

Von Mises stress as a function of deviatoric stress

$\phi^\mathrm{VM}=\sqrt{\frac{3}{2} s_{ij}s_{ij}}$

In [ ]:
def vm_gen_devi(sd):
    """
    Von Mises yield locus in the deviatoric space.
    """
    vm = 0.
    for i in xrange(6):
        vm = vm + sd[i]**2
    return np.sqrt(1.5*vm)

## Von Mises stress in Plane-stress space
$\phi^\mathrm{VM}(\mathbf{\sigma})=[\sigma_{11}^2-\sigma_{11}\sigma_{22}+\sigma_{22}^2]^\frac{1}{2}$


In [ ]:
def vm_ps(s11,s22):
    """
    VM Plane stress (s11,s22)

    Arguments
    ---------
    s11,s22
    
    Returns phi^VM
    """
    y = np.sqrt(s11**2-s11*s22+s22**2)
    return y

## Homogeneous function of degree $k$
$\phi(\alpha \cdot \mathbf{\sigma})=\alpha^k \phi(\mathbf{\sigma})$

$\phi(\mathbf{\sigma})=\frac{\phi(\alpha \cdot \mathbf{\sigma}) }{\alpha^k}$

## Homogeeous function of degree 1.
$\phi(\mathbf{\sigma})=\frac{\phi(\alpha \cdot \mathbf{\sigma}) }{\alpha}$

When $\mathbf{\sigma}=[1,0,0,0,0,0]$

$\phi(\mathbf{\sigma})=Y$

$\phi(\alpha\cdot\sigma)=Y\cdot\alpha$

In [ ]:
def th_2_planestress(th):
    """
    Given theta, return the stress.

    Argument
    --------
    th
    """
    sigma=np.zeros(6)
    sigma[0]=np.cos(th)
    sigma[1]=np.sin(th)    
    return sigma

def th_planestress(th):
    """
    
    Return stress tensors that gives the same
    size (value) of phi
    
    Argument
    --------
    th
    """
    Sigma = th_2_planestress(th)
    y     = vm_gen(Sigma)
    return Sigma / y

## Von Mises yield locus

In [ ]:
nths=1000
ths=np.linspace(-pi,pi,nths)
locus=np.zeros((2,nths))

xy=[]
for i in xrange(len(ths)):
    ys = th_planestress(ths[i]) ## stress tensors
    locus[:,i]=ys[0],ys[1]
    sd = np.zeros(6)
    sd[0],sd[1] = ys[0],ys[1]
    sd = devit(sd)
    x, y = pi_proj(sd)
    xy.append([x,y])
xy=np.array(xy).T
fig=plt.figure(figsize=(7,3))
ax1=fig.add_subplot(121); ax2=fig.add_subplot(122)
ax1.plot(locus[0],locus[1],'k-')
ax2.plot(xy[0],xy[1],'k-')

## deco
lib_dat.pi_rad(ax2,rot=150,rel=1.5)
ys_temp(ax1)

## Associated Flow Rule

- Tensorial representation of associated flow rule

$\dot{\varepsilon}_{ij} = \lambda \frac{\partial \phi(\mathbf{\sigma})}{\partial \sigma_{ij}}$

- 6D vectorial representation

$\dot{\varepsilon}_k=\lambda \frac{\partial \phi(\mathrm{\sigma})}{\partial \sigma_k}$




- Numerical approximation of the above is:

$\dot{\varepsilon}_k=\lambda\ \delta_{ki} \frac{\phi(\sigma_1,\sigma_2,...,\sigma_k+\Delta\sigma_i) - \phi(\sigma_1,\sigma_2,...,\sigma_k+\Delta\sigma_i)}{2\Delta \sigma_i} $

$\Delta \sigma$ should be sufficiently small.

In [ ]:
def assoc_flow(s6=[1,0,0,0,0,0],lamb=1.):
    """
    Argument
    --------
    s6  (6D cauchy stress)
    """
    dlt = 1e-10
    phi = vm_gen(s6)
    s1  = np.zeros(6); s2  = np.zeros(6)
    dki = np.identity(6)
    e_k = np.zeros(6)

    for k in xrange(6):
        dum=0.
        s1=np.zeros(6);
        s2=np.zeros(6);
        for i in xrange(6):
            s1[i] = s[i] + dki[k,i] * dlt
            s2[i] = s[i] - dki[k,i] * dlt
        e_k[k] = lamb*(vm_gen(s1) - vm_gen(s2))/(2*dlt)
    return e_k

## Test Associated Flow Rule

In [ ]:
fig=plt.figure(figsize=(3.5,3))
ax1=fig.add_subplot(111)

nstp=15
ax1.plot(locus[0],locus[1])
for i in xrange(len(locus[0])/nstp):
    x,y = locus[0][i*nstp],locus[1][i*nstp]
    th  = np.arctan2(y,x)
    s   = np.array([x,y,0,0,0,0])

    de = assoc_flow(s,0.02)
    #de[:2]=de[:2]/np.sqrt((de[:2]**2).sum())
    TH  = np.arctan2(de[1],de[0])
    r = 0.2
    dx,dy = r*cos(TH),r*sin(TH)
    ax1.arrow(x,y,dx,dy,color='red')
    ax1.plot(x,y,'.')

ys_temp(ax1)